### Preprocessing OULAD dataset. You may need to download the dataset

In [4]:
from tqdm import tqdm
import pandas as pd
from os.path import join
import math
import seaborn as sns
from scipy import stats
import numpy as np
import random
from collections import defaultdict
import matplotlib.pyplot as plt
import itertools
import operator
from collections import Counter

In [5]:
import sys
import oulad_helper

In [6]:
data_root='root/directory/'

### Student information

In [10]:
student_df_original = pd.read_csv(join(data_root,'studentInfo.csv'))
print('total entries:', len(student_df_original))
print('entry count in each course:')
student_df_original.groupby(['code_module'])[['id_student']].count()

total entries: 32593
entry count in each course:


,id_student
code_module,
AAA,748
BBB,7909
CCC,4434
DDD,6272
EEE,2934
FFF,7762
GGG,2534


In [11]:
student_df_original.groupby('final_result').count()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability
final_result,,,,,,,,,,,
Distinction,3024,3024,3024,3024,3024,3024,2825,3024,3024,3024,3024
Fail,7052,7052,7052,7052,7052,7052,6907,7052,7052,7052,7052
Pass,12361,12361,12361,12361,12361,12361,11830,12361,12361,12361,12361
Withdrawn,10156,10156,10156,10156,10156,10156,9920,10156,10156,10156,10156


In [7]:
'''
Load preprocessed dataframe where some features were converted to numerical types
Note: some entries were removed because a few students took multiple modules.
'''
student_df = pd.read_csv(join(data_root,'student_df_preprocessed.csv'))
print(student_df.shape)
student_df['row_id']=student_df.apply(lambda row: '{}_{}_{}'.format(row.id_student, row.code_module, 
                                                    row.code_presentation), axis=1)
student_df.set_index('row_id',inplace=True, drop=True)
student_df.head(2)

(28785, 20)


,id_student,id_student.1,code_module,code_presentation,id_student.1.1,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,gender_num,disability_num,highest_education_num,final_result_num,age_num,previously_attempted
row_id,,,,,,,,,,,,,,,,,,,,
11391_AAA_2013J,11391,11391,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,0,0,3,1,3,False
28400_AAA_2013J,28400,28400,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,1,0,3,1,1,False


In [9]:
student_df.groupby('final_result').count()

,id_student,id_student.1,code_module,code_presentation,id_student.1.1,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,gender_num,disability_num,highest_education_num,final_result_num,age_num,previously_attempted
final_result,,,,,,,,,,,,,,,,,,,
Distinction,2645,2645,2645,2645,2645,2645,2645,2645,2475,2645,2645,2645,2645,2645,2645,2645,2645,2645,2645
Fail,6264,6264,6264,6264,6264,6264,6264,6264,6135,6264,6264,6264,6264,6264,6264,6264,6264,6264,6264
Pass,10833,10833,10833,10833,10833,10833,10833,10833,10371,10833,10833,10833,10833,10833,10833,10833,10833,10833,10833
Withdrawn,9043,9043,9043,9043,9043,9043,9043,9043,8833,9043,9043,9043,9043,9043,9043,9043,9043,9043,9043


In [13]:
len(oulad_helper.activity_name_dict)

20

In [10]:
student_df.groupby(['age_num'])[['id_student']].count()

,id_student
age_num,
0,20145
1,8462
3,178


In [11]:
'''category to numeric conversion'''
def convert_education(education):
    if education=='No Formal quals':
        return 0
    if education=='Lower Than A Level':
        return 1
    if education=='A Level or Equivalent':
        return 2
    if education=='HE Qualification':
        return 3
    return 4

def convert_result(result):
    if result=='Pass':
        return 1
    if result=='Distinction':
        return 2
    return 0

def convert_age(age):
    if age=='0-35':
        return 0
    if age=='35-55':
        return 1
    return 3
    
# student_df['gender_num'] = student_df.apply(lambda row: 0 if row.gender=='M' else 1, axis=1)
# student_df['disability_num'] = student_df.apply(lambda row: 1 if row.disability=='Y' else 0, axis=1)
# student_df['highest_education_num'] = student_df.apply(lambda row: convert_education(row.highest_education), axis=1)
# student_df['final_result_num'] = student_df.apply(lambda row: convert_result(row.final_result), axis=1)
# student_df['age_num'] = student_df.apply(lambda row: convert_age(row.age_band), axis=1)
# student_df=student_df[~student_df.index.duplicated(keep='first')]

'''convert number of previous attempts into a binary variable'''
# student_df['previously_attempted'] = student_df.apply(lambda row: int(row.num_of_prev_attempts>0), axis=1)
# student_df.to_csv(join(data_root,'student_df_preprocessed.csv'))

# student_df.head()

'convert number of previous attempts into a binary variable'

## Assessment

In [13]:
student_assessment_df=pd.read_csv(join(data_root,'studentAssessment.csv'))
student_assessment_df.set_index(['id_student'],inplace=True)
print(student_assessment_df.shape)
print('assessment was transferred from a previous presentation: ', 
      len(student_assessment_df[student_assessment_df.is_banked==1]))
print('blank score:',len(student_assessment_df[student_assessment_df.is_banked==1]))
student_assessment_df.head()

(173912, 4)
assessment was transferred from a previous presentation:  1909
blank score: 1909


,id_assessment,date_submitted,is_banked,score
id_student,,,,
11391,1752,18,0,78.0
28400,1752,22,0,70.0
31604,1752,17,0,72.0
32885,1752,26,0,69.0
38053,1752,19,0,79.0


In [14]:
student_assessment_df.loc[11391]

,id_assessment,date_submitted,is_banked,score
id_student,,,,
11391,1752,18,0,78.0
11391,1753,53,0,85.0
11391,1754,115,0,80.0
11391,1755,164,0,85.0
11391,1756,212,0,82.0


In [15]:
'''
Calculate the percetage of assessment task completed in each course by each student
and save to file.
'''

def get_assessment_perc(student_id, code_module, code_presentation):
    assessment_count = assessment_count_by_course.loc[(code_module, code_presentation)]
    completed=0
    try:
        if student_id in set(student_assessment_df.index):
            completed = student_assessment_df.loc[student_id].id_assessment.size
    except Exception as e:
        print('student: ', student_id, e)
    return completed*100/assessment_count
    
# assessment_perc_df=student_registration_df[['code_module','code_presentation','id_student']]
# assessment_perc_df['perc_assessment_completed']=assessment_perc_df.apply(lambda row: get_assessment_perc(row.id_student, row.code_module, row.code_presentation), axis=1)
# assessment_perc_df.set_index(['id_student','code_module','code_presentation'], inplace=True)
# assessment_perc_df.to_csv(join(data_root, 'preprocessed', 'assessment_perc_df.csv'))

In [16]:
# assessment_perc_df.head()

### Compute *weekly* activites of different types from StudentVLE

In [19]:
'''dictionary of different activities as defined in oulad_constants.py'''
len(oulad_helper.activity_name_dict), len(oulad_helper.weekly_activity_dict)

(20, 9)

In [14]:
'''Load the preprocessed student-vle dataframe where the
    gender, activity_type, week information were added'''
student_vle = pd.read_csv(join(data_root,'studentVle-preprocessed.csv'))
student_vle = student_vle.loc[:, ~student_vle.columns.str.contains('^Unnamed')]

In [15]:
'''check for null values'''
assert(len(student_vle[student_vle['sum_click'].isnull()]) == 0)

In [23]:
len(set(student_vle.id_student))

26074

In [16]:
print(student_vle.shape)
student_vle.head(2)

(10655280, 10)


,code_module,code_presentation,id_student,id_site,date,sum_click,gender,activity_type,vle_material,week
0,AAA,2013J,28400,546652,-10,4,F,forumng,546652,-1
1,AAA,2013J,28400,546652,-10,1,F,forumng,546652,-1


In [ ]:
'''
Create pivot dataset (combinedly for all courses):
 for each week
     - for each type of activity (e.g., video)
         - number of sessions for that activity (a session is simply one entry in the 
             dataframe for certain action type)
         - total number of clicks for that activity
         
    - add total number of sessions and clicks for the whole week, i.e., aggregated over 
    different types of activities
'''

df = student_vle[student_vle.activity_type.isin(list(oulad_helper.weekly_activity_dict.keys()))]
print('number of entries for the desired activities: ', len(df))
df['row_id']=df.apply(lambda row: '{}_{}_{}'.format(row.id_student, 
                                        row.code_module,row.code_presentation), axis=1)

df.set_index('row_id', inplace=True)
df=df.loc[list(set(df.index).intersection(set(student_df.index)))]

'''aggregate number of sessions and total clicks per week individually for each activity_type'''
weekly_data=df.groupby(['row_id','week', 'activity_type']).agg(
        session_count = ('sum_click','count'), #num of sessions in a week for each activity type
        total_clicks =('sum_click','sum') #total clicks in a week for each activity type
    )

'''convert aggregated values (computed in the previous step) to columns'''
weekly_activities= weekly_data.reset_index().pivot_table(
    columns=['week','activity_type'],
    index='row_id',
    values=['session_count','total_clicks'],
    fill_value=0).reset_index().set_index('row_id')

'''rename columns'''
weekly_activities.columns = ['week_{}_{}_{}'.format(s2,s3,s1) for (s1,s2,s3) in    \
                                               weekly_activities.columns.tolist()]

'''Compute *total* number of sessions and clicks per week for *all* activity types'''
weekly_data=weekly_data.reset_index().groupby(['row_id','week']).agg(
    all_session_week=('session_count','sum'), #total sessions in a week
    all_clicks_week=('total_clicks','sum') #total clicks in a week
    ).reset_index()

'''convert aggregated values (computed in the previous step) to columns'''
weekly_total_pivot= weekly_data.pivot_table(columns='week', index='row_id', 
                            values=['all_session_week','all_clicks_week'], 
                            fill_value=0).reset_index().set_index('row_id')
'''rename columns'''
weekly_total_pivot.columns = ['{}_{}'.format(s1,s2) for (s1,s2) in \
                              weekly_total_pivot.columns.tolist()]

'''append columns'''
weekly_activities=weekly_activities.join(weekly_total_pivot)

In [143]:
weekly_activities['id_student']= weekly_activities.apply(lambda row: int(row.name.split('_')[0]), axis=1)
weekly_activities['id_module']= weekly_activities.apply(lambda row: row.name.split('_')[1], axis=1)
weekly_activities['id_presentation']= weekly_activities.apply(lambda row: row.name.split('_')[2], axis=1)
weekly_activities['gender_num']= student_df.gender_num
weekly_activities['gender_num']= student_df.gender_num
weekly_activities['final_result_num']= weekly_activities.apply(lambda row: 
                                                int(student_df.loc[row.name].final_result_num==0), axis=1)
weekly_activities['credits']=student_df.studied_credits
weekly_activities['highest_education']=student_df.highest_education_num
weekly_activities['previously_attempted']=student_df.previously_attempted
weekly_activities['age_num']=weekly_activities.apply(lambda row: 
                                                     int(student_df.loc[row.name].age_num==0), axis=1)

In [148]:
weekly_activities.groupby(['age_num']).count()

,week_-3_dataplus_session_count,week_-3_forumng_session_count,week_-3_resource_session_count,week_-3_url_session_count,week_-2_dataplus_session_count,week_-2_forumng_session_count,week_-2_htmlactivity_session_count,week_-2_ouwiki_session_count,week_-2_page_session_count,week_-2_resource_session_count,...,all_session_week_38,all_session_week_39,id_student,id_module,id_presentation,gender_num,final_result_num,credits,highest_education,previously_attempted
age_num,,,,,,,,,,,,,,,,,,,,,
0,7716,7716,7716,7716,7716,7716,7716,7716,7716,7716,...,7716,7716,7716,7716,7716,7716,7716,7716,7716,7716
1,17529,17529,17529,17529,17529,17529,17529,17529,17529,17529,...,17529,17529,17529,17529,17529,17529,17529,17529,17529,17529


In [149]:
weekly_activities.to_csv(join(data_root, 'preprocessed-weekly', 'weekly-activities-combined.csv'))

In [36]:
weekly_activities = pd.read_csv(join(data_root, 'preprocessed-weekly', 'weekly-activities-combined.csv'))
weekly_activities.set_index('row_id', inplace=True)
weekly_activities.head(2)

,week_-3_dataplus_session_count,week_-3_forumng_session_count,week_-3_resource_session_count,week_-3_url_session_count,week_-2_dataplus_session_count,week_-2_forumng_session_count,week_-2_htmlactivity_session_count,week_-2_ouwiki_session_count,week_-2_page_session_count,week_-2_resource_session_count,...,all_session_week_39,id_student,id_module,id_presentation,gender_num,final_result_num,credits,highest_education,previously_attempted,age_num
row_id,,,,,,,,,,,,,,,,,,,,,
100064_FFF_2013J,0,0,0,0,0,0,0,0,1,0,...,0,100064,FFF,2013J,1,0,60,2,False,0
100282_BBB_2013J,0,0,0,0,0,0,0,0,0,0,...,0,100282,BBB,2013J,1,1,120,1,True,1


### Percentage of vle items interacted

In [25]:
vle=pd.read_csv(join(data_root,'vle.csv'))
vle.set_index('id_site',inplace=True)
vle.shape

(6364, 5)

In [26]:
activity_types = vle.activity_type.unique()

In [27]:
'''number of vle materials of each type in each course'''
vle_materials = vle.reset_index().groupby(['code_module','code_presentation','activity_type']).id_site.count()
vle_materials.head()

code_module  code_presentation  activity_type
AAA          2013J              dataplus          4
                                forumng          15
                                glossary          2
                                homepage          1
                                oucollaborate     2
Name: id_site, dtype: int64

In [28]:
'''compute number of vle materials in each course offering'''
vle_material_count=student_vle.groupby(['code_module','code_presentation', 
                     'id_student', 'activity_type']).vle_material.unique()#.count()
vle_material_count=vle_material_count.to_frame().reset_index()

vle_material_count['row_id']=vle_material_count.apply(lambda row: 
        '{}_{}_{}'.format(row.id_student, row.code_module, row.code_presentation), axis=1)

vle_material_count.set_index('row_id', inplace=True)

vle_material_count['vle_material_count'] = vle_material_count.apply(lambda row:
                                                    len(row.vle_material), axis=1)

vle_material_count.head()

,code_module,code_presentation,id_student,activity_type,vle_material,vle_material_count
row_id,,,,,,
11391_AAA_2013J,AAA,2013J,11391,forumng,"[546652, 546659, 546657, 546645, 546647, 54665...",11
11391_AAA_2013J,AAA,2013J,11391,homepage,[546614],1
11391_AAA_2013J,AAA,2013J,11391,oucontent,"[546669, 546662, 546668, 546670, 546671, 54666...",32
11391_AAA_2013J,AAA,2013J,11391,resource,"[547009, 546986, 547013, 546907, 547017]",5
11391_AAA_2013J,AAA,2013J,11391,subpage,"[546879, 546876, 546874, 546871]",4


In [29]:
'''compute the percentage of vle materials interacted with by students'''
vle_material_count['perc_interacted'] = vle_material_count.apply(
    lambda row: (100*row.vle_material_count)/(vle_materials.loc[(row.code_module, 
                row.code_presentation, row.activity_type)]+.000001), axis=1)

vle_material_count.head()

,code_module,code_presentation,id_student,activity_type,vle_material,vle_material_count,perc_interacted
row_id,,,,,,,
11391_AAA_2013J,AAA,2013J,11391,forumng,"[546652, 546659, 546657, 546645, 546647, 54665...",11,73.333328
11391_AAA_2013J,AAA,2013J,11391,homepage,[546614],1,99.999900
11391_AAA_2013J,AAA,2013J,11391,oucontent,"[546669, 546662, 546668, 546670, 546671, 54666...",32,47.058823
11391_AAA_2013J,AAA,2013J,11391,resource,"[547009, 546986, 547013, 546907, 547017]",5,5.263158
11391_AAA_2013J,AAA,2013J,11391,subpage,"[546879, 546876, 546874, 546871]",4,66.666656


In [30]:
'''create columns for the percentages of vle material interactions for each activity (e.g., forum) '''
interaction_percentage = vle_material_count.reset_index().pivot_table(
    columns='activity_type', values='perc_interacted', index='row_id', fill_value=0)

interaction_percentage.reset_index(inplace=True)
interaction_percentage.set_index('row_id', inplace=True)

interaction_percentage.columns = ['perc_interacted_{}'.format(s1) for s1 in \
                              interaction_percentage.columns.tolist()]
print(interaction_percentage.shape)
interaction_percentage.head(2)#.columns

(29228, 20)


,perc_interacted_dataplus,perc_interacted_dualpane,perc_interacted_externalquiz,perc_interacted_folder,perc_interacted_forumng,perc_interacted_glossary,perc_interacted_homepage,perc_interacted_htmlactivity,perc_interacted_oucollaborate,perc_interacted_oucontent,perc_interacted_ouelluminate,perc_interacted_ouwiki,perc_interacted_page,perc_interacted_questionnaire,perc_interacted_quiz,perc_interacted_repeatactivity,perc_interacted_resource,perc_interacted_sharedsubpage,perc_interacted_subpage,perc_interacted_url
row_id,,,,,,,,,,,,,,,,,,,,
100064_FFF_2013J,59.999988,99.999983,0.0,99.9999,57.142849,0.0,99.9999,0.0,6.25,84.112149,0.0,62.499992,38.461537,78.571423,99.999991,0.0,22.794117,0.0,74.545453,20.454545
100282_BBB_2013J,0.000000,0.000000,0.0,0.0000,10.526315,0.0,99.9999,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.000000,19.999996,0.0,0.423729,0.0,7.894737,6.666666


#### Create dataframe for semester level features 

##### Compute *semester-level* session count and total clicks for each type of activity 

The total click features were also used in **Waheed et al. paper**

In [31]:
'''
    Compute total number of clicks for each activity type
'''
agg_data = student_vle.groupby(['code_module', 'code_presentation','id_student', 'activity_type']).agg(
                session_count = ('sum_click','count'), 
                sum_click =('sum_click','sum'))
agg_data.reset_index(inplace=True, drop=False)
agg_data['row_id']=agg_data.apply(lambda row: '{}_{}_{}'.format(row.id_student, row.code_module, 
                                                    row.code_presentation), axis=1)
agg_data.set_index('row_id', inplace=True)
agg_data = agg_data.loc[list(set(student_df.index).intersection(set(agg_data.index)))] #remove students who took more than one courses
agg_data.head(2)

,code_module,code_presentation,id_student,activity_type,session_count,sum_click
row_id,,,,,,
79461_CCC_2014B,CCC,2014B,79461,forumng,34,163
79461_CCC_2014B,CCC,2014B,79461,homepage,45,146


In [32]:
semester_activities_df = agg_data.pivot_table(
            columns=['activity_type'], index='row_id', 
            values=['session_count', 'sum_click'], 
#             values=['sum_click'],
            fill_value=0).reset_index().set_index('row_id') 
semester_activities_df.columns = ['{}_{}'.format(s1,s2) for (s1,s2) in    \
                                               semester_activities_df.columns.tolist()]

print(semester_activities_df.shape)
semester_activities_df.head(2)

(25737, 40)


,session_count_dataplus,session_count_dualpane,session_count_externalquiz,session_count_folder,session_count_forumng,session_count_glossary,session_count_homepage,session_count_htmlactivity,session_count_oucollaborate,session_count_oucontent,...,sum_click_ouelluminate,sum_click_ouwiki,sum_click_page,sum_click_questionnaire,sum_click_quiz,sum_click_repeatactivity,sum_click_resource,sum_click_sharedsubpage,sum_click_subpage,sum_click_url
row_id,,,,,,,,,,,,,,,,,,,,,
100064_FFF_2013J,8,10,0,5,239,0,170,0,2,576,...,0,59,25,34,705,0,68,0,590,63
100282_BBB_2013J,0,0,0,0,5,0,5,0,0,0,...,0,0,0,0,1,0,1,0,3,1


#### Add coverage features (i.e., perc. of vle materials interacted with)

In [34]:
'''First remove students for which we do not have coverage information'''
common_idx = set(interaction_percentage.index).intersection(semester_activities_df.index)
print(len(common_idx))
semester_activities_df = semester_activities_df.loc[common_idx]

'''now, copy coverage features'''
print(interaction_percentage.shape, semester_activities_df.shape)
semester_activities_df=semester_activities_df.join(interaction_percentage)
semester_activities_df.shape

25737
(29228, 20) (25737, 40)


(25737, 60)

#### Find *blocks* (=continious weekly activities without a break) of activities 

In [37]:
def find_nonzero_block_len(array):
    if len(array)==0:
        return []
    array = (array > 0.).astype(np.int_)
    
    blocks = [[i for i,value in it] for key,it in \
              itertools.groupby(enumerate(array), key=operator.itemgetter(1)) if key != 0]
    return [len(block) for block in blocks]

'''columns containing weekly session numbers: [all_session_week_n] '''
weekly_session_cols = [x for x in list(weekly_activities.columns) if 'all_session_week_' in x]

'''For each row (i.e., student-course pair or row_id), 
    find consecutive blocks of weeks when that student had at least one session per week'''
block_lens = weekly_activities.apply(lambda row: 
                    find_nonzero_block_len(row[weekly_session_cols].values), axis=1)

block_lens

row_id
100064_FFF_2013J                         [38, 2]
100282_BBB_2013J                             [3]
100561_DDD_2014J          [1, 2, 5, 13, 2, 4, 1]
100621_CCC_2014B                          [1, 1]
1006742_FFF_2014B                           [10]
                                ...             
99670_FFF_2014B              [11, 2, 4, 4, 1, 1]
99799_BBB_2014B      [7, 2, 3, 1, 1, 2, 1, 4, 1]
998493_AAA_2014J                            [42]
999174_FFF_2013B                         [35, 1]
99993_FFF_2013B                    [22, 5, 2, 5]
Length: 25245, dtype: object

##### Add block statistics as semester-level features

In [39]:
'''First remove students for which we do not have sessionblock information'''
common_idx = set(interaction_percentage.index).intersection(block_lens.index)
print(len(common_idx))
semester_activities_df = semester_activities_df.loc[common_idx]

semester_activities_df['blocks'] =  block_lens#.loc[row.name]
semester_activities_df['block_count'] = semester_activities_df.apply(lambda row: len(row.blocks), axis=1)
semester_activities_df['max_block_len'] = semester_activities_df.apply(lambda row: np.max(row.blocks), axis=1)
semester_activities_df['min_block_len'] = semester_activities_df.apply(lambda row: np.min(row.blocks), axis=1)
semester_activities_df['mean_block_len'] = semester_activities_df.apply(lambda row: np.mean(row.blocks), axis=1)
semester_activities_df['var_block_len'] = semester_activities_df.apply(lambda row: np.var(row.blocks), axis=1)

print(semester_activities_df.shape)
semester_activities_df.head(2)

25245
(25245, 66)


,session_count_dataplus,session_count_dualpane,session_count_externalquiz,session_count_folder,session_count_forumng,session_count_glossary,session_count_homepage,session_count_htmlactivity,session_count_oucollaborate,session_count_oucontent,...,perc_interacted_resource,perc_interacted_sharedsubpage,perc_interacted_subpage,perc_interacted_url,blocks,block_count,max_block_len,min_block_len,mean_block_len,var_block_len
row_id,,,,,,,,,,,,,,,,,,,,,
79461_CCC_2014B,0,0,0,0,34,0,45,0,0,0,...,17.948718,0.0,28.571428,15.384614,"[1, 2, 1, 1, 1, 3]",6,3,1,1.500000,0.583333
2356809_BBB_2013J,0,0,0,0,139,2,95,0,3,8,...,11.016949,0.0,42.105262,26.666665,"[10, 4, 11, 2, 4, 1]",6,11,1,5.333333,14.555556


In [42]:
len(block_lens), len(semester_activities_df), len(weekly_activities)

(25245, 25245, 25245)

In [43]:
semester_activities_df['gender_num'] = student_df.gender_num
semester_activities_df['final_result_num']= semester_activities_df.apply(lambda row: 
                            0 if student_df.loc[row.name].final_result_num==0 else 1, axis=1)

semester_activities_df['credits']=student_df.studied_credits
semester_activities_df['highest_education']=student_df.highest_education_num
semester_activities_df['previously_attempted']=student_df.previously_attempted
semester_activities_df['age_num']=weekly_activities.apply(lambda row: 
                                                     int(student_df.loc[row.name].age_num==0), axis=1)
semester_activities_df.fillna(0, inplace=True)
semester_activities_df.to_csv(join(data_root, 'semester-level-activity.csv'))
semester_activities_df.head(2)

,session_count_dataplus,session_count_dualpane,session_count_externalquiz,session_count_folder,session_count_forumng,session_count_glossary,session_count_homepage,session_count_htmlactivity,session_count_oucollaborate,session_count_oucontent,...,max_block_len,min_block_len,mean_block_len,var_block_len,gender_num,final_result_num,credits,highest_education,previously_attempted,age_num
row_id,,,,,,,,,,,,,,,,,,,,,
79461_CCC_2014B,0,0,0,0,34,0,45,0,0,0,...,3,1,1.500000,0.583333,0,0,90,2,False,1
2356809_BBB_2013J,0,0,0,0,139,2,95,0,3,8,...,11,1,5.333333,14.555556,1,1,60,2,False,0


In [ ]:
list(semester_activities_df.columns)